In [2]:
import torch
import torchtext.vocab as vocab

In [3]:
# 查看支持的向量
vocab.pretrained_aliases.keys()

dict_keys(['charngram.100d', 'fasttext.en.300d', 'fasttext.simple.300d', 'glove.42B.300d', 'glove.840B.300d', 'glove.twitter.27B.25d', 'glove.twitter.27B.50d', 'glove.twitter.27B.100d', 'glove.twitter.27B.200d', 'glove.6B.50d', 'glove.6B.100d', 'glove.6B.200d', 'glove.6B.300d'])

In [5]:
# 下载glove 预训练的词向量
cache_dir="../data/pretrain_models"
# glove = vocab.pretrained_aliases["glove.6B.50d"](cache=cache_dir)
glove = vocab.GloVe(name='6B', dim=50, cache=cache_dir) 

100%|█████████▉| 399999/400000 [00:09<00:00, 43260.72it/s]


返回的实例glove主要有以下三个属性：

stoi: 词到索引的字典：
itos: 一个列表，索引到词的映射；
vectors: 词向量。

# 运用词向量求近义词

In [7]:
# glove 通过余弦相似度求解近义词

def knn(W,x,k):
    cos=torch.matmul(W,x.view((-1,)))/((torch.sum(W * W, dim=1) + 1e-9).sqrt() * torch.sum(x * x).sqrt())
    _, topk = torch.topk(cos, k=k)
    topk=topk.cpu().numpy()
    #  返回topk的角标（在词向量中的位置）以及相似度得分
    return topk, [cos[i].item() for i in topk]

def get_similar_tokens(query_token, k, embed):
    topk, cos = knn(embed.vectors,embed.vectors[embed.stoi[query_token]], k+1)
    for i, c in zip(topk[1:], cos[1:]):  # 除去输入词
        print('cosine sim=%.3f: %s' % (c, (embed.itos[i])))

In [8]:
get_similar_tokens('chip', 3, glove)

cosine sim=0.856: chips
cosine sim=0.749: intel
cosine sim=0.749: electronics


# 运用词向量求类比词
对于类比关系中的4个词 a:b::c:d，给定前3个词a、b和c，求d。设词w的词向量为vec(w)。求类比词的思路是，搜索与vec(c)+vec(b)−vec(a)的结果向量最相似的词向量。

In [11]:
def get_analogy(token_a,token_b,token_c,embedd):
    # 获取词向量
    vecs = [embedd.vectors[embedd.stoi[t]] for t in [token_a, token_b, token_c]]
    # 求解差值
    x = vecs[1] - vecs[0] + vecs[2]
    topk, cos = knn(embedd.vectors, x, 1)
    return embedd.itos[topk[0]]

In [12]:
get_analogy('man', 'woman', 'son', glove)

'daughter'